<a href="https://colab.research.google.com/github/sumanlaraee/fyp_Genrative_AI_LLMS_IDEFICS/blob/main/WristMinderPro_rogue_before_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rouge-score nltk


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d25476f32bb538fedf47fe80723447c6b364e11751d2a754ca69a39e37df490c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -i https://pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
  Installing build dependencies .

In [ ]:
!pip install --upgrade transformers

In [ ]:
pip install accelerate --upgrade


In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, DataCollatorForTokenClassification, Trainer, TrainingArguments, BitsAndBytesConfig
import torchvision.transforms as transforms
import pandas as pd
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:


device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = "HuggingFaceM4/idefics-9b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"],
)


In [ ]:
from transformers import AutoProcessor

In [ ]:

processor = AutoProcessor.from_pretrained(checkpoint)
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="auto")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:

df = pd.read_csv("/content/ReportWithImage.csv")
df = df.drop_duplicates()
df.to_json('train.jsonl', orient='records', lines=True)

train_dataset = load_dataset('json', data_files='/content/train.jsonl', split="train")


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:

data = {
    'User': [],
    'Assistant': []
}
imageUrl = []

for example in train_dataset:
    user_text = f"User: This is a Picture of Wrist Xray, Is It Fracture or Not in Xray?\nImage: https://github.com/GoutamSachdev/ReportDataSet/blob/main/Match_fracture/{example['image_name']}?raw=true \n<end_of_utterance>"
    assistant_text = f"\nAssistant: {example['indication']}. Findings: {example['findings']}. Impression: {example['impression']}\nResult: Wrist Xray is {example['Problems']}<end_of_utterance>"
    imageUrl.append(f"https://github.com/GoutamSachdev/ReportDataSet/blob/main/Match_fracture/{example['image_name']}?raw=true")
    data['User'].append(user_text)
    data['Assistant'].append(assistant_text)

prompts = []
count = 0
size = min(len(data['User']), len(data['Assistant']))

for i in range(size):
    if count >= 0:
        pair = [data['User'][i], data['Assistant'][i]]
        count = count + 1
    else:
        break
    prompts.append(pair)


In [ ]:

def do_inference(model, processor, prompt, max_new_tokens=150):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids
    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    input = processor(prompt, return_tensors="pt").to(device)
    generated_ids = model.generate(
        **input,
        eos_token_id=[eos_token_id],
        bad_words_ids=bad_words_ids,
        max_new_tokens=max_new_tokens,
        early_stopping=True)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)
    return generated_text


In [ ]:

from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training


In [ ]:

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


In [ ]:

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none"
)

In [ ]:
model = get_peft_model(model, peft_config)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_side="right",)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_side="right")
tokenizer.pad_token = tokenizer.eos_token
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN = tokenizer.eos_token
def format_dataset(examples):
    image_name = examples['image_name']
    prompt = alpaca_prompt.format(examples["instructions"], f"{examples['inputs']}\nhttps://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/{image_name}?raw=true", examples["outputs"]) + EOS_TOKEN
    tokens = tokenizer(prompt, truncation=True, max_length=256, padding="max_length",)
    tokens["labels"] = tokens['input_ids'].copy()
    return tokens


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

dataset = load_dataset('csv', data_files='/content/format Report.csv', split="train")
dataset = dataset.map(format_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1236 [00:00<?, ? examples/s]

In [ ]:
def convert_to_rgb(image_path):
    image = Image.open(image_path)
    if image.mode == "RGB":
        return image
    image_rgba = image.convert("RGBA")
    background = Image.new("RGBA", image_rgba.size, (255, 255, 255))
    alpha_composite = Image.alpha_composite(background, image_rgba)
    return alpha_composite.convert("RGB")

In [ ]:
def tmp_transforms(examples):
    image_size = processor.image_processor.image_size
    image_mean = processor.image_processor.image_mean
    image_std = processor.image_processor.image_std
    image_transform = transforms.Compose([
        convert_to_rgb,
        transforms.RandomResizedCrop((image_size, image_size), scale=(0.9, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=image_mean, std=image_std)
    ])
    prompts = []
    for i in range(len(examples['instructions'])):
        img = examples['image_name'][i]
        text = alpaca_prompt.format(examples["instructions"], f"""{examples["inputs"]}\nhttps://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/{img}?raw=true""", examples["outputs"]) + EOS_TOKEN
        prompts.append(text)
    inputs = processor(prompts, transform=image_transform, return_tensors="pt").to(device)
    inputs["labels"] = inputs["input_ids"]
    return inputs

In [ ]:

tmp = dataset.train_test_split(test_size=0.3)
train_dataset = tmp["train"]
test_dataset = tmp["test"]
train_dataset.set_transform(tmp_transforms)
test_dataset.set_transform(tmp_transforms)


In [ ]:
def evaluate_model(model, processor, test_dataset):
    predictions = []
    references = []
    for example in test_dataset:
        prompt = tokenizer.decode(example['input_ids'], skip_special_tokens=True)
        generated_text = do_inference(model, processor, prompt)
        predictions.append(generated_text)
        references.append(tokenizer.decode(example['labels'], skip_special_tokens=True))

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    bleu_scores = []

    for ref, pred in zip(references, predictions):
        rouge = scorer.score(ref, pred)
        for key in rouge_scores:
            rouge_scores[key] += rouge[key].fmeasure

        bleu_scores.append(sentence_bleu([ref.split()], pred.split()))

    rouge_scores = {key: value / len(references) for key, value in rouge_scores.items()}
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)

    return rouge_scores, avg_bleu_score

# Evaluate before training
pre_train_rouge, pre_train_bleu = evaluate_model(model, processor, test_dataset)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
["['cat','pronatorsign','fraction','occurs']"]
https://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/0421_1100503522_01_WR-L2_F006.png?raw=true

###Response:
['.Softtissueswellingnoted..WristGrayisFracture']
https://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/0421_1100503522_01_WR-L2_F006.png?raw=true

The following is a list of the most common errors in the use of the word “that”:

The word “that” is used to introduce a subordinate clause.

The word “that” is used to introduce a subordinate clause.

The word “that” is used to introduce a subordinate clause.

The word “that” is used to introduce a subordinate clause.

The word “that”
Belowisaninstructionthatdescribesatask,pairedwithaninputthatprovidesfurthercontext.Writearesponsethatappropriatelycompletestherequest.

###Instruction:
['BasedontheanalysisofWristGray,thefollowingissueshavebeenidentified:']

###Input:
["['softtisse','socissue','fracture','fracture','t

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
print(f"ROUGE scores before training: {pre_train_rouge}")
print(f"BLEU score before training: {pre_train_bleu}")


ROUGE scores before training: {'rouge1': 0.37519097113622185, 'rouge2': 0.2577526038331184, 'rougeL': 0.2936289316108826}
BLEU score before training: 1.2336149388856005e-80
